# Create H, A, T

In [1]:
import pandas as pd
df = pd.read_csv('./datasets/raw/tdec.csv')

In [2]:
dfs = df.sample(100, random_state=99)

In [3]:
import stanza

nlp = stanza.Pipeline(
    lang="id",
    processors="tokenize,pos,lemma",
    download_method=stanza.DownloadMethod.REUSE_RESOURCES,
)

d:\repo\lda-bertopic\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-19 11:43:27 WARNING: Language id package default expects mwt, which has been added
2023-08-19 11:43:28 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-08-19 11:43:28 INFO: Using device: cpu
2023-08-19 11:43:28 INFO: Loading: tokenize
2023-08-19 11:43:28 INFO: Loading: mwt
2023-08-19 11:43:28 INFO: Loading: pos
2023-08-19 11:43:28 INFO: Loading: lemma
2023-08-19 11:43:28 INFO: Done loading processors!


In [4]:
from time import time

_HAS_t = {}

t_start = time()
H = dfs.filter(['headline'], axis=1)
H.columns = ['H']

_HAS_t['H'] = time() - t_start
A = dfs.filter(['body'], axis=1)
A.columns = ['A']
A['A'] = dfs['headline'] + '. ' + A['A']

# _HAS_t['A'] = time() - _HAS_t['H']
# S = dfs.filter(['body'], axis=1)
# S.columns = ['S']
# S['S'] = dfs['headline'] + '. ' + S['S']
# in_S = list(S['S'].apply(lambda h: stanza.Document([], text=h)))
# out_S = nlp(in_S)
# S = pd.DataFrame([s.text for a in out_S for s in a.sentences], columns=['S'])

# _HAS_t['S'] = time() - _HAS_t['A']

In [5]:
from utils import e_variant

HAS_t = pd.DataFrame(pd.Series(e_variant()), columns=['variant'])

d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
d:\repo\lda-bertopic\.venv\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The 'nopython'

In [6]:
r = []
for v in HAS_t['variant']:
    r.append(_HAS_t[v[0]])
HAS_t['segmentation'] = pd.Series(r)

In [7]:
HAS_t

,variant,segmentation
0,HWN,0.001003


In [8]:
# HAS = pd.concat([H, A, S], axis=1)
HAS = pd.concat([H, A], axis=1)

In [9]:
HAS_t.to_parquet('./results/HAS_t.parquet')
HAS.to_parquet('./results/HAS.parquet')

# Create Variants

In [10]:
import pandas as pd

HAS = pd.read_parquet('./results/HAS.parquet')
HAS_t = pd.read_parquet('./results/HAS_t.parquet')

In [11]:
import stanza

nlp = stanza.Pipeline(
    lang="id",
    processors="tokenize,pos,lemma",
    download_method=stanza.DownloadMethod.REUSE_RESOURCES,
)

2023-08-19 11:43:38 WARNING: Language id package default expects mwt, which has been added
2023-08-19 11:43:39 INFO: Loading these models for language: id (Indonesian):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |

2023-08-19 11:43:39 INFO: Using device: cpu
2023-08-19 11:43:39 INFO: Loading: tokenize
2023-08-19 11:43:39 INFO: Loading: mwt
2023-08-19 11:43:39 INFO: Loading: pos
2023-08-19 11:43:39 INFO: Loading: lemma
2023-08-19 11:43:39 INFO: Done loading processors!


In [12]:
from gensim.models.phrases import Phrases

allowed_pos = ['NOUN', 'PROPN', 'VERB', 'X']

def preprocess_clw(out_df, c=False, l=False, w=False, n=False):
    r = [] # result
    if n:
        r_t = []
    for d in out_df:
        tokens = []
        for s in d.sentences:
            for token in s.words:
                _token = token.text
                if w:
                    if token.upos not in allowed_pos:
                        continue
                if l:
                    _token = token.lemma if token.lemma else _token
                if c:
                    _token = _token if l else token.text.lower()
                tokens.append(_token)
        r.append(' '.join(tokens))
        if n:
            r_t.append(tokens)
    if n:
        bigram = Phrases(r_t).freeze()
        trigram = Phrases(bigram[r_t]).freeze()
        r = [' '.join(d) for d in trigram[bigram[r_t]]]
    return r

In [13]:
from time import time
from utils import e_variant


def preprocess():
    t_start = time()
    nlp_datasets = {}
    for v in e_variant():
        df = HAS[v[0]].dropna()
        in_doc = list(df.apply(lambda x: stanza.Document([], text=x)))
        out_doc = nlp(in_doc)
        nlp_datasets[v[0]] = out_doc
    t_base = time() - t_start
    r = {} # result
    r_time = []
    variant = e_variant()
    for v in variant:
        t_preprocess = time()
        process_C = 'C' in v
        process_L = 'L' in v
        process_W = 'W' in v
        process_N = 'N' in v
        r[v] = pd.Series(preprocess_clw(
            nlp_datasets[v[0]],
            c=process_C,
            l=process_L,
            w=process_W,
            n=process_N
        ))
        r_time.append(time() - t_preprocess + t_base)
    HAS_t['preprocess'] = pd.Series(r_time)
    return pd.DataFrame(r)

In [14]:
HAS_p = preprocess()

In [15]:
HAS_p
HAS_t

,variant,segmentation,preprocess
0,HWN,0.001003,2.34099


In [16]:
HAS_p.to_parquet('./results/HAS_p.parquet')
HAS_t.to_parquet('./results/HAS_t.parquet')